# Liskov Substitution Principle (LSP)

## Definition

Objects of a superclass should be replaceable with objects of a subclass without affecting the correctness of the program.

This principle ensures that a subclass can stand in for its superclass without causing unexpected behavior. It emphasizes the need for subclasses to adhere to the behavior expected by the superclass, maintaining integrity in the inheritance hierarchy.

In other words, if class B is a subclass of class A, then instances of A should be replaceable with instances of B without altering the desirable properties of the program (correctness, task performed, etc.).

# Example 01

we have a base class PaymentProcessor and two subclasses CreditCardProcessor and BitcoinProcessor. The PaymentProcessor class has a method process_payment that each subclass should implement.

In [ ]:

class PaymentProcessor:
    def process_payment(self, amount):
        raise NotImplementedError("Subclasses should implement this!")

class CreditCardProcessor(PaymentProcessor):
    def process_payment(self, amount):
        return f"Processing credit card payment of ${amount}"

class BitcoinProcessor(PaymentProcessor):
    def process_payment(self, amount):
        raise Exception("Bitcoin payment processing is not supported")

def handle_payment(processor: PaymentProcessor, amount):
    return processor.process_payment(amount)

# Create payment processors
credit_card_processor = CreditCardProcessor()
bitcoin_processor = BitcoinProcessor()

# Handle payments
print(handle_payment(credit_card_processor, 100))  # Output: Processing credit card payment of $100
print(handle_payment(bitcoin_processor, 100))  # Raises Exception: Bitcoin payment processing is not supported




Issues:

1. Semantic Misrepresentation: The BitcoinProcessor subclass inherits from PaymentProcessor but cannot actually process Bitcoin payments. This creates a semantic mismatch where a subclass is expected to behave like its superclass (PaymentProcessor) but cannot fulfill that contract.

2. Runtime Errors: When a BitcoinProcessor instance is passed to a function expecting a PaymentProcessor, such as handle_payment, it throws an exception (Exception: Bitcoin payment processing is not supported). This breaks the program flow and can lead to unexpected behavior.

3. Violation of Client Expectations: Clients using the PaymentProcessor superclass expect that any subclass (CreditCardProcessor, BitcoinProcessor, etc.) can be used interchangeably. However, BitcoinProcessor fails this expectation by throwing an exception instead of processing payments like other subclasses.

4. Maintenance Challenges: Future modifications to add more payment processors might lead to similar issues if subclasses do not adhere to the expected behavior defined in the superclass (PaymentProcessor).

Consequences:

1. Code Fragility: The code becomes fragile because client code (handle_payment function or any code expecting a PaymentProcessor) cannot reliably substitute any subclass without potential runtime errors.

2. Reduced Flexibility: Introducing new payment processors (e.g., EthereumProcessor, BankTransferProcessor) that may behave differently could further complicate the system and potentially break existing client code.

3. Difficulty in Testing and Debugging: Testing becomes challenging because the behavior of subclasses (BitcoinProcessor) deviates from the expected behavior defined by the superclass (PaymentProcessor), leading to inconsistencies and hard-to-predict outcomes.



## Solution 01

To adhere to Liskov Substitution Principle (LSP), we should redesign the system to ensure that all subclasses (CreditCardProcessor, BitcoinProcessor, etc.) of PaymentProcessor can substitute the superclass without altering the expected behavior. This can be achieved by ensuring that:

1. Each subclass (CreditCardProcessor, BitcoinProcessor, etc.) properly implements the process_payment method according to its specific logic.
2. Client code (handle_payment function, etc.) should be able to rely on the fact that any subclass of PaymentProcessor can be used interchangeably without causing errors or unexpected behaviors.

In [ ]:
class PaymentProcessor:
    def process_payment(self, amount):
        raise NotImplementedError("Subclasses should implement this!")

class CreditCardProcessor(PaymentProcessor):
    def process_payment(self, amount):
        return f"Processing credit card payment of ${amount}"

class BitcoinProcessor(PaymentProcessor):
    def process_payment(self, amount):
        return f"Processing Bitcoin payment of ${amount}"

class UnsupportedPaymentProcessor(PaymentProcessor):
    def process_payment(self, amount):
        raise Exception("This payment method is not supported")


1. CreditCardProcessor and BitcoinProcessor properly implement the process_payment method according to their specific payment processing logic.

2. UnsupportedPaymentProcessor explicitly raises an exception, indicating that this payment method is not supported.

3. Client code (handle_payment function or any code expecting a PaymentProcessor) can now rely on the fact that any subclass of PaymentProcessor adheres to the expected behavior, thus maintaining the system's consistency and adhering to LSP.

The violation of Liskov Substitution Principle (LSP) in the initial payment processing example led to semantic mismatches, runtime errors, and a lack of client expectations. By correcting the design to ensure that all subclasses of PaymentProcessor adhere to the expected behavior, we can create a more reliable and maintainable system where subclasses can be substituted without causing unexpected behaviors or errors.

# Example 02

### Without Adhering to LSP

In [ ]:
class Rectangle:
    def __init__(self, width, height):
        self.width = width
        self.height = height

    def set_width(self, width):
        self.width = width

    def set_height(self, height):
        self.height = height

    def get_area(self):
        return self.width * self.height

class Square(Rectangle):
    def __init__(self, side):
        super().__init__(side, side)

    def set_width(self, width):
        self.width = width
        self.height = width

    def set_height(self, height):
        self.width = height
        self.height = height


In this design, Square inherits from Rectangle. However, it violates the LSP because a Square cannot behave like a Rectangle in all cases. For instance, setting the width of a Square changes both its width and height, which is not the expected behavior for a Rectangle.

### Adhering to LSP

To adhere to LSP, we can refactor the code so that the Square and Rectangle classes are siblings rather than inheriting one from the other.

In [ ]:
from abc import ABC, abstractmethod

class Shape(ABC):
    @abstractmethod
    def get_area(self):
        pass

class Rectangle(Shape):
    def __init__(self, width, height):
        self.width = width
        self.height = height

    def set_width(self, width):
        self.width = width

    def set_height(self, height):
        self.height = height

    def get_area(self):
        return self.width * self.height

class Square(Shape):
    def __init__(self, side):
        self.side = side

    def set_side(self, side):
        self.side = side

    def get_area(self):
        return self.side * self.side


def print_area(shape):
    print(f"The area is: {shape.get_area()}")

rectangle = Rectangle(5, 10)
square = Square(5)

print_area(rectangle)  # The area is: 50
print_area(square)     # The area is: 25

In this design, Rectangle and Square both inherit from the Shape abstract base class, but they do not inherit from each other. This way, each class can maintain its own behavior without violating the LSP.



# Example 03

consider a system for managing various types of bank accounts. We have a base class BankAccount and two derived classes: SavingsAccount and CurrentAccount.



### Without Adhering to LSP

In [ ]:
class BankAccount:
    def __init__(self, balance):
        self.balance = balance

    def deposit(self, amount):
        self.balance += amount

    def withdraw(self, amount):
        if amount > self.balance:
            raise ValueError("Insufficient funds")
        self.balance -= amount

class SavingsAccount(BankAccount):
    def withdraw(self, amount):
        if amount > self.balance:
            raise ValueError("Insufficient funds")
        self.balance -= amount

class CurrentAccount(BankAccount):
    def withdraw(self, amount):
        self.balance -= amount


In this design, SavingsAccount and CurrentAccount both inherit from BankAccount. However, CurrentAccount allows overdraft (withdrawals that can exceed the balance), which violates the LSP. Substituting a BankAccount with a CurrentAccount could lead to unexpected behavior.

### Adhering to LSP

In [ ]:
from abc import ABC, abstractmethod

class BankAccount(ABC):
    def __init__(self, balance):
        self.balance = balance

    def deposit(self, amount):
        self.balance += amount

    @abstractmethod
    def withdraw(self, amount):
        pass

class SavingsAccount(BankAccount):
    def withdraw(self, amount):
        if amount > self.balance:
            raise ValueError("Insufficient funds")
        self.balance -= amount

class CurrentAccount(BankAccount):
    def withdraw(self, amount):
        self.balance -= amount

# Using the Classes

def process_withdrawal(account, amount):
    account.withdraw(amount)
    print(f"Withdrawal successful. New balance: {account.balance}")

savings = SavingsAccount(100)
current = CurrentAccount(100)

process_withdrawal(savings, 50)  # Withdrawal successful. New balance: 50
process_withdrawal(current, 150) # Withdrawal successful. New balance: -50


The BankAccount class defines an abstract withdraw method, ensuring that all derived classes must implement this method. The SavingsAccount class enforces no overdraft by checking the balance, while the CurrentAccount class allows overdraft. Both classes adhere to the LSP because they conform to the interface defined by BankAccount.

# Summary
By following the Liskov Substitution Principle:

We ensure that subclasses can replace their base class without altering the desirable properties of the program.
We create a more maintainable and flexible system where classes can be extended without unexpected behavior.
We define clear contracts in the base class, which subclasses must adhere to, ensuring consistent behavior across different types.